In [14]:
# scrape code

import requests
import pandas as pd
from time  import sleep
import datetime
import json

pd.options.mode.chained_assignment = None  # default='warn'
# scrapelist = ["fi", "gr", "hu", "ie", "lt", "li", "lu", "mk"]
scrapelist = ["gr"]   # country specific doesnt work somehow

print('start')
print(datetime.datetime.now())

for country in scrapelist:
    for page in range(1, 1):
        r = requests.get(
                "https://www.vivino.com/api/explore/explore",
                params = {
                    "country_code": country,
                    #"country_codes[]":"pt",
                    "currency_code":"EUR",
                    "grape_filter":"varietal",
                    "min_rating":"1",
                    "order_by":"price",
                    "order":"desc",
                    "page": page,
                    "price_range_max":"500",
                    "price_range_min":"0",
                    "wine_type_ids[]":["1", "2", "3"]
                },
                headers= {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
                }
            )
        if r.status_code == 429:
            print('429 Error; sleeping 15min')
            sleep(15*60)

        matches_found = len(r.json()["explore_vintage"]["matches"])

        if matches_found == 0:
            print('finished')
            break

        else:
            feature_list = [ 'Winery', 'Name', 'Year', 'Type', 'Country', 'Region',
                             'Mean Rating', 'No. Ratings', 'Price',
                             'acidity', 'fizziness', 'intensity', 'sweetness', 'tannin',
                             'flavor notes', 'food notes',
                             'grape - 1', 'grape - 2', 'grape - 3' ]

            dataframe    = pd.DataFrame(0., index   = range(matches_found),
                                           columns = feature_list)
            i = 0

            for idx, t in enumerate(r.json()["explore_vintage"]["matches"]):
                try:
                    dataframe.at[idx, 'Winery']    = t["vintage"]["wine"]["winery"]["name"]
        
                    dataframe.at[idx, 'Name']      = t["vintage"]["wine"]["name"]

                    dataframe.at[idx, 'Year']      = int(t["vintage"]["year"])

                    type_id                        = int(t["vintage"]["wine"]["type_id"])
                    
                    if   type_id == 1:
                        dataframe.at[idx, 'Type'] = 'Red'
                    elif type_id == 2:
                        dataframe.at[idx, 'Type'] = 'White'
                    elif type_id == 3:
                        dataframe['Type'][i] = 'Sparkling'
                    elif type_id == 4:
                        dataframe['Type'][i] = 'Rose'
                    else:
                        dataframe['Type'][i] = 'NI'
                except:
                    pass



                try:
                    dataframe['Country'][i]    = t["vintage"]["wine"]["region"]["country"]["name"]
                except:
                    dataframe['Country'][i]    = 'NA'
                try:
                    dataframe['Region'][i]     = t["vintage"]["wine"]["region"]["name"]
                except:
                    dataframe['Region'][i]     = 'NA'
                try:
                    dataframe['Mean Rating'][i] = t["vintage"]["statistics"]["ratings_average"]
                except:
                    dataframe['Mean Rating'][i] = 'NA'
                try:
                    dataframe['No. Ratings'][i] = t["vintage"]["statistics"]["ratings_count"]
                except:
                    dataframe['No. Ratings'][i] = 'NA'
                
                ID                         = t["vintage"]["id"]
                dataframe['Price'][i]      = ID

                    # for now screw prices or get 429HTTP Request Error, Price API request:
                    # price_r = requests.get( "https://www.vivino.com/api/checkout_prices",
                    #                            params = {"vintage_id": ID,
                    #                                      "currency_code": "EUR"},
                    #                            headers= {
                    # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"
                    #                             }
                    #                            )
                    # dataframe['Price'][i]  = price_r.json()["prices"]["availability"]["median"]['amount']

                if pd.isnull(dataframe['Price'][i]):
                        dataframe['Price'][i]  = 'NA'
                except:
                    dataframe['Price'][i]      = 'NA'


                if t["vintage"]["wine"]["taste"]["structure"] is not None:                 # TASTE STRUCTURE

                    dataframe['acidity'][i]   = t["vintage"]["wine"]["taste"]["structure"]["acidity"]
                    dataframe['fizziness'][i] = t["vintage"]["wine"]["taste"]["structure"]["fizziness"]
                    dataframe['intensity'][i] = t["vintage"]["wine"]["taste"]["structure"]["intensity"]
                    dataframe['sweetness'][i] = t["vintage"]["wine"]["taste"]["structure"]["sweetness"]
                    dataframe['tannin'][i]    = t["vintage"]["wine"]["taste"]["structure"]["tannin"]

                    if pd.isnull(dataframe['fizziness'][i]):
                        dataframe['fizziness'][i] = 'NA'


                if t["vintage"]["wine"]["taste"]["flavor"] is not None:                    # FLAVOUR NOTES
                    notes = []
                    for j in range(0,len(t["vintage"]["wine"]["taste"]["flavor"])):
                        notes.append(t["vintage"]["wine"]["taste"]["flavor"][j]["group"])
                    separator = '; '
                    dataframe['flavor notes'][i] = separator.join(notes)

                if t["vintage"]["wine"]["style"] and t["vintage"]["wine"]["style"]["food"] is not None:
                    foods = []
                    for j in range(0,len(t["vintage"]["wine"]["style"]["food"])):
                        foods.append(t["vintage"]["wine"]["style"]["food"][j]["name"])
                    separator = '; '
                    dataframe['food notes'][i] = separator.join(foods)

                try:
                        if len(t["vintage"]["wine"]["style"]["grapes"]) == 1:
                            dataframe['grape - 1'][i] = t["vintage"]["wine"]["style"]["grapes"][0]['name'] 
                        if len(t["vintage"]["wine"]["style"]["grapes"]) == 2:
                            dataframe['grape - 1'][i] = t["vintage"]["wine"]["style"]["grapes"][0]['name']
                            dataframe['grape - 2'][i] = t["vintage"]["wine"]["style"]["grapes"][1]['name']
                        if len(t["vintage"]["wine"]["style"]["grapes"]) == 3:
                            dataframe['grape - 1'][i] = t["vintage"]["wine"]["style"]["grapes"][0]['name']
                            dataframe['grape - 2'][i] = t["vintage"]["wine"]["style"]["grapes"][1]['name']
                            dataframe['grape - 3'][i] = t["vintage"]["wine"]["style"]["grapes"][2]['name']
                except:  
                        dataframe['grape - 1'][i] = 'NA'
                        dataframe['grape - 2'][i] = 'NA'
                        dataframe['grape - 3'][i] = 'NA'

                i += 1

        sleep(3)
        if page == 1:
            wineframe = dataframe
        else:
            wineframe = wineframe.append(dataframe, ignore_index = True)
    print('fin with', country)
    print(datetime.datetime.now())
print('fin')
print(datetime.datetime.now())

start
2020-08-10 22:13:21.166218


ValueError: setting an array element with a sequence.

In [ ]:
# save as csv:

wineframe.to_csv("ENTERNAME")

In [10]:
# display json respones:

print(json.dumps(r.json(), indent=4, sort_keys=True))

{
    "e": "eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660TS9SS7Z1DQ1SK7A1NFFLT7MtSyzKTC1JzFHLT7ItKMpMTlXLt01JLU5WKy-JjgWaAKaMIJQxAHOoG00=",
    "explore_vintage": {
        "bottle_type_errors": [],
        "market": {
            "country": "gr",
            "currency": {
                "code": "EUR",
                "name": "Euros",
                "prefix": "\u20ac",
                "suffix": null
            }
        },
        "matches": [
            {
                "price": null,
                "prices": [],
                "vintage": {
                    "grapes": null,
                    "has_valid_ratings": true,
                    "id": 3938524,
                    "image": {
                        "location": "//images.vivino.com/thumbs/AM656VTfS1ueJqZBxmaHWQ_pl_480x640.png",
                        "variations": {
                            "bottle_large": "//images.vivino.com/thumbs/AM656VTfS1ueJqZBxmaHWQ_pb_x960.png",
                            "bottle_medium": "//images.v